In [126]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


%matplotlib qt5

# data preparing
point_count = 20
points = np.random.rand(point_count * 2).reshape((point_count, 2))
plt.scatter(points[:, 0], points[:, 1])
plt.show()

穷尽搜索算法

对于点集上的任意的一条直线，如果其他所有点都在这条直线的一侧，则这两个点在凸包的点集内。<br>
其中判断点在直线的哪侧可以用两条直线的叉积的正负值决定。 <br>
算法基本思想：<br>
遍历[原点集]对应的所有直线，如果其他点都在点集的一侧，则添加这两个点到[凸包点集]中。

In [2]:
epsilon = 1e-5

LEFT_SIDE = 1
RIGHT_SIDE = 2
INLINE_SIDE = 0
def point_side(x1, y1, x2, y2, x3, y3):
    """
    @return 1: 右侧， 2：左侧， 0：在同一条直线
    """
    dot_value = x1 * y2 + x2 * y3 + x3 * y1 - x1 * y3 - x2 * y1 - x3 * y2
    if dot_value > epsilon:
        return LEFT_SIDE
    elif dot_value < epsilon:
        return RIGHT_SIDE
    return INLINE_SIDE

In [3]:
convex_hull_lines = []

if point_count <= 2:
    print("input point size must be great than 2")
for i in range(point_count):
    for j in range(i + 1, point_count):
        previous_side = INLINE_SIDE
        allSame = True
        for k in range(point_count):
            if k != i and k != j:
                cur_side = point_side(points[i][0], points[i][1], points[j][0], points[j][1], points[k][0], points[k][1])
                if cur_side == previous_side or previous_side == INLINE_SIDE or cur_side == INLINE_SIDE:
                    previous_side = cur_side
                    continue
                else:
                    allSame = False
                    break
        if allSame:
            convex_hull_lines.append((points[i], points[j]))

convex_hull_lines

[(array([0.90010028, 0.22906409]), array([0.91245884, 0.79083504])),
 (array([0.90010028, 0.22906409]), array([0.80885278, 0.04285968])),
 (array([0.91245884, 0.79083504]), array([0.88552795, 0.96848521])),
 (array([0.88552795, 0.96848521]), array([0.34154526, 0.93788143])),
 (array([0.34154526, 0.93788143]), array([0.02697326, 0.66737167])),
 (array([0.02697326, 0.66737167]), array([0.00639313, 0.45892407])),
 (array([0.00639313, 0.45892407]), array([0.32524678, 0.09553586])),
 (array([0.74976554, 0.02988612]), array([0.80885278, 0.04285968])),
 (array([0.74976554, 0.02988612]), array([0.32524678, 0.09553586]))]

In [4]:
plt.scatter(points[:, 0], points[:, 1])
for line in convex_hull_lines:
    plt.plot([line[0][0], line[1][0]], [line[0][1], line[1][1]], color='red')
plt.show()

In [5]:
# 将线转化成点的集合
def point_to_hash(point):
    return str(point[0]) +','+ str(point[1])
left_hash = point_to_hash(convex_hull_lines[0][0])
points_dict = {left_hash: convex_hull_lines[0][1]}
right_hashes = [point_to_hash(convex_hull_lines[0][1])]
for line in convex_hull_lines[1:]:
    if point_to_hash(line[0]) in points_dict.keys() or point_to_hash(line[1]) in right_hashes:
        points_dict[point_to_hash(line[1])] = line[0]
        right_hashes.append(point_to_hash(line[0]))
    else:
        points_dict[point_to_hash(line[0])] = line[1]
        right_hashes.append(point_to_hash(line[1]))

convex_hull_points = [convex_hull_lines[0][0], convex_hull_lines[0][1]]
point_hash = point_to_hash(convex_hull_points[1])
step = 1
step_count = len(points_dict)
while step < step_count:
    next_point = points_dict[point_hash]
    convex_hull_points.append(next_point)
    point_hash = point_to_hash(next_point)
    step += 1

convex_hull_points

[array([0.90010028, 0.22906409]),
 array([0.91245884, 0.79083504]),
 array([0.88552795, 0.96848521]),
 array([0.34154526, 0.93788143]),
 array([0.02697326, 0.66737167]),
 array([0.00639313, 0.45892407]),
 array([0.32524678, 0.09553586]),
 array([0.74976554, 0.02988612]),
 array([0.80885278, 0.04285968]),
 array([0.90010028, 0.22906409])]

In [6]:
chps = np.array(convex_hull_points)
fig, ax = plt.subplots()
ax.scatter(points[:, 0], points[:, 1])
line, = plt.plot([], [], color='red')
def update(draw_points):
    line.set_data(chps[0:draw_points, 0], chps[0:draw_points, 1])
    return line,
ani = animation.FuncAnimation(fig, update, frames=np.arange(0, len(chps)+1), interval=500, repeat=True, repeat_delay=2000)
plt.show()

In [9]:
# using ArtistAnimation 
# fig, ax = plt.subplots()
# ax.scatter(points[:,0], points[:,1])

# plots = []
# for chp in convex_hull_lines:
#     plot, = plt.plot([chp[0][0], chp[1][0]], [chp[0][1], chp[1][1]], color='red')
#     plots.append([plot])
# ani = animation.ArtistAnimation(fig, plots, interval=500)
# plt.show()

In [133]:
point_count = 40
points = np.random.rand(point_count * 2).reshape((point_count, 2))

In [134]:
# points with minimum y must be in the convex hull points.
# sort all points by angles. a simpler solutions:
# split points into two groups by point's x coordinate is greater or small than min_y's x
# higher points sorted by its' y ascend and smaller points sorted by its' y descend.
# combine these points.
point_minY = points[points[:, 1].argmin()][1]
points_minY = points[points[:,1]==point_minY]
points_minY = sorted(points_minY, key=lambda p: p[0], reverse=True)
p0 = points_minY[0]

points_right = points[points[:, 0] > p0[0]]
points_middle = points[points[:, 0] == p0[0]]
points_left = points[points[:, 0] < p0[0]]

def angle(p0, p1):
    return np.arccos((p1[1] - p0[1])/np.sqrt((p1[0]-p0[0])**2 + (p1[1]-p0[1])**2))

points_right = np.array(sorted(points_right, key=lambda p: -angle(p0, p)))
points_middle = np.array(sorted(points_middle, key=lambda p: p[1]))
points_left = np.array(sorted(points_left, key=lambda p: angle(p0, p)))
print("middle points:\n", points_middle)
print("left points:\n", points_left)
print("right points:\n", points_right)

middle points:
 [[0.24911198 0.0537784 ]]
left points:
 [[0.1491935  0.86675398]
 [0.19271733 0.44552498]
 [0.0456004  0.94907106]
 [0.03911204 0.8724709 ]
 [0.05860872 0.72341903]
 [0.11814678 0.4282176 ]
 [0.24292403 0.07022977]
 [0.16876434 0.24612723]
 [0.15041063 0.2473838 ]
 [0.16908403 0.17043462]
 [0.07103547 0.12481972]]
right points:
 [[0.43911136 0.06244422]
 [0.88232596 0.21687006]
 [0.77758925 0.22635329]
 [0.75763614 0.23610491]
 [0.73634067 0.230163  ]
 [0.49185819 0.17413785]
 [0.64609587 0.25276115]
 [0.85150788 0.40352943]
 [0.74892024 0.3545353 ]
 [0.90101104 0.4690596 ]
 [0.66219079 0.32533606]
 [0.90909567 0.4999313 ]
 [0.83852728 0.68094133]
 [0.92278984 0.87357076]
 [0.94359884 0.91334258]
 [0.79646225 0.75175681]
 [0.54096778 0.48740329]
 [0.58652733 0.63375405]
 [0.37940435 0.31185016]
 [0.59570076 0.77273721]
 [0.58025709 0.80103022]
 [0.54171578 0.77865087]
 [0.45720836 0.58878964]
 [0.57145127 0.90271426]
 [0.57484957 0.95385413]
 [0.41724058 0.56694244]
 [0

In [135]:
convex_hull_points = [p0]

fig, ax = plt.subplots()

plt.scatter(points[:, 0], points[:, 1])
plt.show()
for point_cur in points_right:
    while len(convex_hull_points) > 1:
        point_last = convex_hull_points[-1]
        point_last2 = convex_hull_points[-2]
        if point_side(point_cur[0], point_cur[1], point_last2[0], point_last2[1], point_last[0], point_last[1]) == RIGHT_SIDE:
            convex_hull_points.pop(-1)
        else:
            break
    convex_hull_points.append(point_cur)

if len(points_middle) > 1:
    convex_hull_points.append(points_middle[-1])

for point_cur in points_left:
    while len(convex_hull_points) > 1:
        point_last = convex_hull_points[-1]
        point_last2 = convex_hull_points[-2]
        if point_side(point_cur[0], point_cur[1], point_last2[0], point_last2[1], point_last[0], point_last[1]) == RIGHT_SIDE:
            convex_hull_points.pop(-1)
        else:
            break
    convex_hull_points.append(point_cur)

for p in convex_hull_points:
    plt.scatter(p[0], p[1], c='r', marker='x')
plt.show()